<a href="https://colab.research.google.com/github/nurikahraman/FoodTracker/blob/main/YoloNAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p '/content/dataset/val'
!mkdir -p '/content/dataset/train'

#Datasets (This gets downloaded in "/content/foodChallenge/data")
!echo "🗄 Preparing the validation dataset..."
!unzip -q "/content/drive/MyDrive/Colab Notebooks/content/FoodRecognitionChallenge/val.zip" -d "/content/dataset/val"

!echo "🗄 Preparing the dataset for training..."
!unzip -q "/content/drive/MyDrive/Colab Notebooks/content/FoodRecognitionChallenge/train.zip" -d "/content/dataset/train"


In [ ]:
!pip install super-gradients

In [2]:
from IPython.display import clear_output
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training import Trainer, dataloaders, models
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

[2024-04-29 19:49:30] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-04-29 19:49:39] INFO - utils.py - NumExpr defaulting to 2 threads.


In [17]:
class config:
    #eğitim parametreleri
    CHECKPOINT_DIR = '/content/drive/MyDrive/Colab Notebooks/Aptal Buzdolabı/outputs'# checkpoint klasörünün tam yolu (absolute path)
    EXPERIMENT_NAME = 'Aptal Buzdolabı' # eğitim sürecine verdiğimiz isim

    #veri seti parametreleri
    DATA_DIR = '/content/dataset/' # veri setinin bulunduğu ana klasör

    TRAIN_IMAGES_DIR = 'train/images' # DATA_DIR içinde eğitim verilerinin bulunduğu klasörün yolu
    TRAIN_LABELS_DIR = 'train' # DATA_DIR içinde eğitim verilerinin etiketlerinin bulunduğu klasörün yolu

    VAL_IMAGES_DIR = 'val/images' # DATA_DIR içinde validasyon verilerinin bulunduğu klasörün yolu
    VAL_LABELS_DIR = 'val' # DATA_DIR içinde validasyon verilerinin etiketlerinin  bulunduğu klasörün yolu

    #TEST_IMAGES_DIR = 'test/images' # DATA_DIR içinde test verilerinin bulunduğu klasörün yolu
    #TEST_LABELS_DIR = 'test/labels' # DATA_DIR içinde test verilerinin etiketlerinin  bulunduğu klasörün yolu
    CLASSES = ['food'] # sınıfların isimleri.

    NUM_CLASSES = len(CLASSES)

    #ver-yükleyici (dataloader) parametreleri
    DATALOADER_PARAMS = {
      'batch_size':16,
      'num_workers':2
    }

    # model parametreleri
    MODEL_NAME = 'yolo_nas_l' # yolo_nas_s, yolo_nas_m, yolo_nas_l arasından birini seçiyoruz
    PRETRAINED_WEIGHTS = 'coco' # burada tek seçenek var: 'coco'

#from pycocotools.coco import COCO
#class Paths:
#  DATASET_DIR = "/content/dataset/"
#  TRAIN_DATA_DIR = f"{DATASET_DIR}train/"
#  TRAIN_IMAGES_DIR = f"{TRAIN_DATA_DIR}images/"
#  TRAIN_ANNOTATIONS = f"{TRAIN_DATA_DIR}annotations.json"
#  VAL_DATA_DIR = f"{DATASET_DIR}val/"
#  VAL_ANNOTATIONS = f"{VAL_DATA_DIR}annotations.json"
#  VAL_IMAGES_DIR = f"{VAL_DATA_DIR}images/"


#class DatasetLabels:
#  TRAIN = "dataset_train"
#  VAL = "dataset_val"

#train_coco = COCO(Paths.TRAIN_ANNOTATIONS)

In [18]:
trainer = Trainer(experiment_name=config.EXPERIMENT_NAME, ckpt_root_dir=config.CHECKPOINT_DIR)

In [19]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.VAL_IMAGES_DIR,
        'labels_dir': config.VAL_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TEST_IMAGES_DIR,
        'labels_dir': config.TEST_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

[2024-04-29 20:12:48] WARNING - yolo_format_detection.py - 24119 images are note associated to any label file
[2024-04-29 20:12:48] WARNING - yolo_format_detection.py - As a consequence, 0/24119 images and 0/0 label files will be used.


ValueError: _setup_data_source() should return the number of available samples but got 0